In [ ]:
"""
	Autores:
	- Astrid Alvarado
	- Junior Lara
"""

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, col, lower, avg, round, count, sum, when, first, explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType

In [5]:
spark = SparkSession\
    .builder\
    .appName("Solver problema 3")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/28 21:52:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
# Para esta primera parte basta con inferencia para extraer la data necesaria.
# Se necesitan los siguientes campos
# - title
# - gender
# - popularity

file = "../tests/vshort-final_animedataset.csv"

DF_FINAL_DATASET = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

DF_FINAL_DATASET.show(5)

+--------+--------+--------+-------+------+-------------+----+------+-----+---------+------+----------+--------------------+
|username|anime_id|my_score|user_id|gender|        title|type|source|score|scored_by|  rank|popularity|               genre|
+--------+--------+--------+-------+------+-------------+----+------+-----+---------+------+----------+--------------------+
|karthiga|      21|       9|2255153|Female|    One Piece|  TV| Manga| 8.54|   423868|  91.0|        35|Action, Adventure...|
|karthiga|      59|       7|2255153|Female|      Chobits|  TV| Manga| 7.53|   175388|1546.0|       188|Sci-Fi, Comedy, D...|
|karthiga|      74|       7|2255153|Female| Gakuen Alice|  TV| Manga| 7.77|    33244| 941.0|      1291|Comedy, School, S...|
|karthiga|     120|       7|2255153|Female|Fruits Basket|  TV| Manga| 7.77|   167968| 939.0|       222|Slice of Life, Co...|
|karthiga|     178|       7|2255153|Female| Ultra Maniac|  TV| Manga| 7.26|     9663|2594.0|      2490|Magic, Comedy, Ro...|


In [ ]:
# Por DataFrame

# Calcular la cantidad de usuarios por género y la popularidad para cada título
DF_count_female_male_by_title = DF_FINAL_DATASET.groupBy("title") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count"),      # Contar los males
               first("popularity").alias("popularity")                          # Tomar la popularidad única
           )

# Agregar las columnas de porcentaje para hombres y mujeres
DF_count_female_male_by_title_with_percen = DF_count_female_male_by_title.withColumn(
    "female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
DF_count_female_male_by_title_with_percen.show(truncate=False)

+---------------------------------------------------+------------+----------+----------+-----------------+---------------+
|title                                              |female_count|male_count|popularity|female_percentage|male_percentage|
+---------------------------------------------------+------------+----------+----------+-----------------+---------------+
|"""Bungaku Shoujo"" Movie"                         |1           |0         |1556      |100.0            |0.0            |
|"Gyakuten Saiban: Sono ""Shinjitsu""               |1           |1         |5839.0    |50.0             |50.0           |
|"Naruto Shippuuden: Shippuu! ""Konoha Gakuen"" Den"|1           |0         |1771      |100.0            |0.0            |
|.hack//Roots                                       |0           |1         |1391      |0.0              |100.0          |
|.hack//Sign                                        |0           |1         |650       |0.0              |100.0          |
|.hack//Tasogare

In [30]:
# Por SQL

# Crear una vista temporal para el DataFrame
DF_FINAL_DATASET.createOrReplaceTempView("final_dataset")

# Consulta SQL para calcular la cantidad de usuarios por género, la popularidad y los porcentajes
SQL_count_female_male_by_title = spark.sql("""
    SELECT 
        title,
        COUNT(CASE WHEN gender = 'Female' THEN 1 END) AS female_count,
        COUNT(CASE WHEN gender = 'Male' THEN 1 END) AS male_count,
        FIRST(popularity) AS popularity,
        ROUND(COUNT(CASE WHEN gender = 'Female' THEN 1 END) * 100.0 / 
              (COUNT(CASE WHEN gender = 'Female' THEN 1 END) + COUNT(CASE WHEN gender = 'Male' THEN 1 END)), 2) AS female_percentage,
        ROUND(COUNT(CASE WHEN gender = 'Male' THEN 1 END) * 100.0 / 
              (COUNT(CASE WHEN gender = 'Female' THEN 1 END) + COUNT(CASE WHEN gender = 'Male' THEN 1 END)), 2) AS male_percentage
    FROM 
        final_dataset
    GROUP BY 
        title
""")

# Mostrar los resultados
SQL_count_female_male_by_title.show(truncate=False)

+---------------------------------------------------+------------+----------+----------+-----------------+---------------+
|title                                              |female_count|male_count|popularity|female_percentage|male_percentage|
+---------------------------------------------------+------------+----------+----------+-----------------+---------------+
|"""Bungaku Shoujo"" Movie"                         |1           |0         |1556      |100.00           |0.00           |
|"Gyakuten Saiban: Sono ""Shinjitsu""               |1           |1         |5839.0    |50.00            |50.00          |
|"Naruto Shippuuden: Shippuu! ""Konoha Gakuen"" Den"|1           |0         |1771      |100.00           |0.00           |
|.hack//Roots                                       |0           |1         |1391      |0.00             |100.00         |
|.hack//Sign                                        |0           |1         |650       |0.00             |100.00         |
|.hack//Tasogare

In [ ]:
# Para esta segunda parte basta con inferencia para extraer la data necesaria.
# Se necesitan los siguientes campos
# - gender
# - genre

# Por DataFrame

# Separar los géneros en la columna "genre" y explotar la lista en filas individuales
df_exploded_genres = DF_FINAL_DATASET.withColumn("genre", explode(split(col("genre"), ", ")))

# Contar la cantidad de usuarios por género (Female y Male) para cada género de anime
DF_count_female_male_by_genres = df_exploded_genres.groupBy("genre") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count")       # Contar los males
           )


# Agregar las columnas de porcentaje para hombres y mujeres con redondeo a 2 decimales
DF_count_female_male_by_genres_with_percen = DF_count_female_male_by_genres.withColumn(
    "female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
DF_count_female_male_by_genres_with_percen.show(truncate=False)

+------------+------------+----------+-----------------+---------------+
|genre       |female_count|male_count|female_percentage|male_percentage|
+------------+------------+----------+-----------------+---------------+
|Seinen      |82          |16        |83.67            |16.33          |
|Romance     |241         |58        |80.6             |19.4           |
|Dementia    |18          |7         |72.0             |28.0           |
|Magic       |60          |26        |69.77            |30.23          |
|Thriller    |40          |15        |72.73            |27.27          |
|Josei       |21          |4         |84.0             |16.0           |
|Adventure   |124         |78        |61.39            |38.61          |
|Super Power |64          |45        |58.72            |41.28          |
|Martial Arts|23          |6         |79.31            |20.69          |
|Sports      |29          |7         |80.56            |19.44          |
|Shoujo      |74          |11        |87.06        

In [32]:
# Por SQL

# Crear una vista temporal para el DataFrame
DF_FINAL_DATASET.createOrReplaceTempView("final_dataset")

# Consulta SQL para calcular la cantidad de usuarios por género y los porcentajes para cada género
SQL_count_female_male_by_genres = spark.sql("""
    SELECT 
        genre,
        COUNT(CASE WHEN gender = 'Female' THEN 1 END) AS female_count,
        COUNT(CASE WHEN gender = 'Male' THEN 1 END) AS male_count,
        ROUND(COUNT(CASE WHEN gender = 'Female' THEN 1 END) * 100.0 / 
              (COUNT(CASE WHEN gender = 'Female' THEN 1 END) + COUNT(CASE WHEN gender = 'Male' THEN 1 END)), 2) AS female_percentage,
        ROUND(COUNT(CASE WHEN gender = 'Male' THEN 1 END) * 100.0 / 
              (COUNT(CASE WHEN gender = 'Female' THEN 1 END) + COUNT(CASE WHEN gender = 'Male' THEN 1 END)), 2) AS male_percentage
    FROM (
        SELECT 
            gender,
            EXPLODE(SPLIT(genre, ', ')) AS genre
        FROM 
            final_dataset
    ) exploded_genres
    GROUP BY 
        genre
""")

# Mostrar los resultados
SQL_count_female_male_by_genres.show(truncate=False)

+------------+------------+----------+-----------------+---------------+
|genre       |female_count|male_count|female_percentage|male_percentage|
+------------+------------+----------+-----------------+---------------+
|Seinen      |82          |16        |83.67            |16.33          |
|Romance     |241         |58        |80.60            |19.40          |
|Dementia    |18          |7         |72.00            |28.00          |
|Magic       |60          |26        |69.77            |30.23          |
|Thriller    |40          |15        |72.73            |27.27          |
|Josei       |21          |4         |84.00            |16.00          |
|Adventure   |124         |78        |61.39            |38.61          |
|Super Power |64          |45        |58.72            |41.28          |
|Martial Arts|23          |6         |79.31            |20.69          |
|Sports      |29          |7         |80.56            |19.44          |
|Shoujo      |74          |11        |87.06        

In [29]:
# Para esta tercera parte basta con inferencia para extraer la data necesaria.
# Se necesitan los siguientes campos
# - source
# - gender

# Por DataFrame

DF_count_female_male_by_source = DF_FINAL_DATASET.groupBy("source") \
           .agg(
               count(when(col("gender") == "Female", 1)).alias("female_count"),  # Contar las females
               count(when(col("gender") == "Male", 1)).alias("male_count")      # Contar los males
           )

#DF_count_female_male_by_source_with_percen

# Agregar las columnas de porcentaje para hombres y mujeres con redondeo a 2 decimales
DF_count_female_male_by_source_with_percen = DF_count_female_male_by_source.withColumn(\
	"female_percentage", round((col("female_count") / (col("female_count") + col("male_count")) * 100), 2)
).withColumn(
    "male_percentage", round((col("male_count") / (col("female_count") + col("male_count")) * 100), 2)
)

# Mostrar los resultados
DF_count_female_male_by_source_with_percen.show()

+------------+------------+----------+-----------------+---------------+
|      source|female_count|male_count|female_percentage|male_percentage|
+------------+------------+----------+-----------------+---------------+
|          TV|           1|         1|             50.0|           50.0|
|Visual novel|          36|        11|             76.6|           23.4|
|    Original|         136|        81|            62.67|          37.33|
|       Novel|          43|        15|            74.14|          25.86|
|     Unknown|          23|         0|            100.0|            0.0|
|       Other|           7|         2|            77.78|          22.22|
|       Manga|         374|       106|            77.92|          22.08|
|4-koma manga|          20|         1|            95.24|           4.76|
|       Music|           1|         1|             50.0|           50.0|
|        Game|           7|         3|             70.0|           30.0|
|   Web manga|          11|         1|            9

In [33]:
# Por SQL

# Crear una vista temporal para el DataFrame
DF_FINAL_DATASET.createOrReplaceTempView("final_dataset")

# Consulta SQL para calcular la cantidad de usuarios por género y los porcentajes para cada source
SQL_count_female_male_by_source = spark.sql("""
    SELECT 
        source,
        COUNT(CASE WHEN gender = 'Female' THEN 1 END) AS female_count,
        COUNT(CASE WHEN gender = 'Male' THEN 1 END) AS male_count,
        ROUND(COUNT(CASE WHEN gender = 'Female' THEN 1 END) * 100.0 / 
              (COUNT(CASE WHEN gender = 'Female' THEN 1 END) + COUNT(CASE WHEN gender = 'Male' THEN 1 END)), 2) AS female_percentage,
        ROUND(COUNT(CASE WHEN gender = 'Male' THEN 1 END) * 100.0 / 
              (COUNT(CASE WHEN gender = 'Female' THEN 1 END) + COUNT(CASE WHEN gender = 'Male' THEN 1 END)), 2) AS male_percentage
    FROM 
        final_dataset
    GROUP BY 
        source
""")

# Mostrar los resultados
SQL_count_female_male_by_source.show()

+------------+------------+----------+-----------------+---------------+
|      source|female_count|male_count|female_percentage|male_percentage|
+------------+------------+----------+-----------------+---------------+
|          TV|           1|         1|            50.00|          50.00|
|Visual novel|          36|        11|            76.60|          23.40|
|    Original|         136|        81|            62.67|          37.33|
|       Novel|          43|        15|            74.14|          25.86|
|     Unknown|          23|         0|           100.00|           0.00|
|       Other|           7|         2|            77.78|          22.22|
|       Manga|         374|       106|            77.92|          22.08|
|4-koma manga|          20|         1|            95.24|           4.76|
|       Music|           1|         1|            50.00|          50.00|
|        Game|           7|         3|            70.00|          30.00|
|   Web manga|          11|         1|            9

In [ ]:
# Para esta cuarta parte basta con inferencia para extraer la data necesaria.
# Se necesitan los siguientes campos
# - 
# - 

In [34]:
spark.stop()